In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import data as a dataframe

freestyle = pd.read_csv('speedrun_data.csv')

#Reorganize indexes
# speed_df.set_index(['Star Rating','Monster Name'],inplace=True)

freestyle.head()

,Star Rating,Monster Name,Quest,Runner,Time,Weapon,Platform,Ruleset
0,6,Alatreon,The Evening Star,DEVA,"01'55""11",Heavy Bowgun,PC,Freestyle
1,6,Alatreon,The Evening Star,DEVA,"02'05""13",Heavy Bowgun,PC,Freestyle
2,6,Alatreon,The Evening Star,Jackdaw갈까마귀,"02'24""56",Heavy Bowgun,PC,Freestyle
3,6,Alatreon,The Evening Star,Jackdaw갈까마귀,"02'30""96",Heavy Bowgun,PC,Freestyle
4,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",Heavy Bowgun,PC,Freestyle


In [3]:
# Filter out 'repeat runs'
# 'Repeat run': run on the same monster, same quest, by the same runner with the same weapon, but different times.
# Keep only the fastest run

freestyle = freestyle.drop_duplicates(['Monster Name','Quest','Runner','Weapon'],keep='first') #Already sorted by time,
                                                                                               #first time = fastest!


In [4]:
#Function to convert the time field to seconds
def to_seconds(time_string):
    
    #Get minutes, seconds and centi seconds from string
    minutes = int(time_string[0:2])
    seconds = int(time_string[3:5])
    cents = int(time_string[6:])
    
    #Return total in seconds
    return 60*minutes+seconds+cents/100
    
#Add Time (Seconds) column
freestyle.insert(5, 'Time (Seconds)', freestyle['Time'].apply(to_seconds))

In [5]:
#Convert Star Rating to string (discrete values!)
freestyle['Star Rating']=freestyle['Star Rating'].apply(str)

In [6]:
#Abbreviate weapon names

weapon_dict = {'Great Sword':'GS',
               'Long Sword':'LS',
               'Sword And Shield':'SnS',
               'Dual Blades':'DB',
               'Lance':'Lance',
               'Gunlance':'GL',
               'Hammer':'Hammer',
               'Hunting Horn':'HH',
               'Switch Axe':'SA',
               'Charge Blade':'CB',
               'Insect Glaive':'IG',
               'Heavy Bowgun':'HBG',
               'Light Bowgun':'LBG',
               'Bow':'Bow'    
               }

#Apply dict to weapons column
freestyle['Weapon']=freestyle['Weapon'].apply(lambda x: weapon_dict[x])

In [7]:
# Sort dataframe by, in order: Star Rating, Monster Name, Quest, and Time
freestyle = freestyle.sort_values(['Star Rating','Monster Name','Quest','Time (Seconds)'],ascending=[False,True,True,True])

#Fix indexes
freestyle = freestyle.reset_index(drop=True)

In [8]:
# Function to generate rankings by quest or monster
def general_rank(speed_df,rank_by_column='Quest'):
    
    ranking=[] #Star with empty list
    
    #Get a list of all items to iterate through
    rank_by_list = speed_df[rank_by_column].unique()
    
    #Iterate through all quests or monsters
    for item in rank_by_list: 
        #First entry is always rank 1
        rank = 1; 

        #Get slice of the input df corresponding to the quest name
        df_slice = speed_df[speed_df[rank_by_column]==item]

        #Append rankings to the list, for the length of the slice
        for index, row in df_slice.iterrows():
            ranking.append(rank)
            rank += 1

        #Reset rank count for next iteration
        rank=1;
    
    return pd.DataFrame(data=ranking,index=range(0,len(ranking)))

In [9]:
freestyle['Quest/General'] = general_rank(freestyle,rank_by_column='Quest')
freestyle['Monster/General'] = general_rank(freestyle,rank_by_column='Monster Name')
freestyle.head(200)

,Star Rating,Monster Name,Quest,Runner,Time,Time (Seconds),Weapon,Platform,Ruleset,Quest/General,Monster/General
0,6,Alatreon,Dawn of the Death Star,Jackdaw갈까마귀,"02'44""40",164.40,HBG,PC,Freestyle,1,1
1,6,Alatreon,Dawn of the Death Star,Relial,"02'58""53",178.53,HBG,PC,Freestyle,2,2
2,6,Alatreon,Dawn of the Death Star,Randel Daora,"03'26""71",206.71,HBG,PC,Freestyle,3,3
3,6,Alatreon,Dawn of the Death Star,HK_MH_OLAY,"03'41""38",221.38,GS,PC,Freestyle,4,4
4,6,Alatreon,Dawn of the Death Star,ブロントさん,"03'52""16",232.16,SnS,PC,Freestyle,5,5
...,...,...,...,...,...,...,...,...,...,...,...
195,6,Alatreon,The Evening Star,LVIS-Gaming,"07'43""86",463.86,CB,PC,Freestyle,63,196
196,6,Alatreon,The Evening Star,KCGemini,"07'49""51",469.51,Lance,PS4,TA Rules,64,197
197,6,Alatreon,The Evening Star,Benji-boi,"07'50""25",470.25,SA,PS4,TA Rules,65,198
198,6,Alatreon,The Evening Star,Fataripper,"07'50""88",470.88,DB,PS4,Freestyle,66,199


In [10]:
# Function to generate rankings by quest or monster, separating each weapon
def weapon_rank(speed_df,rank_by_column='Quest',weapon_column='Weapon'):
    #Start with empty list
    ranking=[] 
    
    #Get a list of all items to iterate through
    rank_by_list = speed_df[rank_by_column].unique()
    
    #Iterate through all quests or monsters
    for item in rank_by_list: 
        
        #Get slice of the input df corresponding to the quest name
        df_slice = speed_df[speed_df[rank_by_column]==item]
    
        #Get list of weapons with an entry for this quest/monster
        weapon_list = df_slice[weapon_column].unique()

        #Iterate through all weapons
        for weapon in weapon_list:
            #First entry is always rank 1
            rank = 1;     

            #Get subslice of slice_df corresponding to this weapon
            df_subslice = df_slice[df_slice[weapon_column]==weapon]

            #Append rankings to the list, for the length of the slice
            for subindex, subrow in df_subslice.iterrows():
                ranking.append(rank)
                rank += 1

            #Reset rank count for next iteration
            rank=1;
    
    return pd.DataFrame(data=ranking,index=range(0,len(ranking)))

In [11]:
#Sort values, but include a sorting by weapon, so a rank can be made by quest and weapon type
freestyle = freestyle.sort_values(['Star Rating','Monster Name','Quest','Weapon','Time (Seconds)'],
                                  ascending=[False,True,True,True,True])
freestyle = freestyle.reset_index(drop=True) #Fix indexes

#Rank by quest and weapon type
freestyle['Quest/Weapon'] = weapon_rank(freestyle,rank_by_column='Quest',weapon_column='Weapon')



#Sort values, sort by weapon before quest, so we can get a rank just on monsters, by weapon type
freestyle = freestyle.sort_values(['Star Rating','Monster Name','Weapon','Quest','Time (Seconds)'],
                                  ascending=[False,True,True,True,True])
freestyle = freestyle.reset_index(drop=True) #Fix indexes

#Ranky by monster and weapon type
freestyle['Monster/Weapon'] = weapon_rank(freestyle,rank_by_column='Monster Name',weapon_column='Weapon')


#Go back to previous sorting
freestyle = freestyle.sort_values(['Star Rating','Monster Name','Quest','Time (Seconds)'],ascending=[False,True,True,True])

#Fix indexes
freestyle = freestyle.reset_index(drop=True)

In [12]:
#Add 4 columns for the 4 rankings:

#Quest/General: rank all runs for the same QUEST consecutively
#Quest/Weapon: rank all runs for the same QUEST by weapon type

#Monster/General: rank all runs for the same monster consecutively
#Monster/Weapon: rank all runs for the same monster by weapon type

# rank_list = ['Quest/General', 'Quest/Weapon','Monster/General', 'Monster/Weapon',]
# freestyle = pd.concat([freestyle,pd.DataFrame(columns=rank_list)],axis=1)
# freestyle

In [13]:
# #Rank by ???/General
# # ??? can be either Quest or Monster Name
# def get_rank_general(df,rank_by_column='Quest',ranked_column='Quest/General'):
#     """
#     Ranks speedruns by the Quest/General criteria: rank all runs for the same QUEST consecutively
#     df: dataframe with speedrun data to rank
#     quest_name: name of quest to rank
#     """
    
#     #Number of rows
#     rows = df.shape[0]
    
#     #Variable to remember current rank
#     current_rank=0;
    
#     for index, row in df.iterrows():
        
#         #Get the current entry to rank by and the previous one
#         current_entry = df.loc[(df.index==index),rank_by_column].to_list()[0] 
#         try:
#             previous_entry = df.loc[(df.index==index-1),rank_by_column].to_list()[0] 
#         except IndexError:
#             previous_entry = ''
            
#         if (index == 0) or (current_entry != previous_entry): #First row of a new quest/monster
#             df.loc[(df.index==0),ranked_column] = 1
#             current_rank = 2
#         else:
#             df.loc[(df.index==index),ranked_column] = current_rank
#             current_rank += 1
            
#     return df[ranked_column]

In [14]:
# #Get the two general ranks done
# freestyle['Quest/General'] = get_rank_general(freestyle,'Quest','Quest/General')
# freestyle['Monster/General'] = get_rank_general(freestyle,'Monster Name','Monster/General')

In [15]:
# # Separate TA runs. Note that Freestyle runs also encompass TA runs (ie a TA run is also a Freestyle run, but a Freestyle run
# # not be a TA run)

# ta = freestyle[freestyle['Ruleset']=='TA Rules'].copy()
# ta = ta.drop('Ruleset',axis=1) #Drop Ruleset column as it is redundant

In [16]:
# #Very early analysis: just get average clear time by weapons

# avg_freestyle = freestyle.groupby('Weapon').mean().reset_index()
# avg_ta = ta.groupby('Weapon').mean().reset_index()

# #Sort
# avg_freestyle = avg_freestyle.sort_values('Time (Seconds)',ascending=True)
# avg_ta = avg_ta.sort_values('Time (Seconds)',ascending=True)

In [17]:
# #Plot average freestyle times
# plt.figure(figsize=(16,7))
# sns.barplot(x='Weapon',y='Time (Seconds)',data=avg_freestyle)

In [18]:
# #Plot average TA times
# plt.figure(figsize=(16,7))
# sns.barplot(x='Weapon',y='Time (Seconds)',data=avg_ta)